In [1]:
!git clone "https://github.com/FinanceData/marcap.git" marcap

fatal: destination path 'marcap' already exists and is not an empty directory.


In [2]:
def today_mc(symbol, today_year):
    """파라미터로 넘겨준 기업 코드의 오늘 기준 시가총액 리턴"""
    
    df = pd.read_csv('marcap/data/marcap-{}.csv.gz'.format(today_year), dtype={'Code':str}, parse_dates=['Date'])
    mc = list(df[df['Code'] == symbol]['Marcap'])
    if len(mc) != 0:
        return mc[-1]
    else:
        return None

In [3]:
def to_float(str):
    """쉼표가 포함된 문자열을 float타입으로 변형"""
    
    result = ''
    num_list = str.split(',')
    for num in num_list:
        result += num
    return float(result)

In [4]:
import FinanceDataReader as fdr
import OpenDartReader
import pandas as pd 
import numpy as np
import datetime

pd.set_option('display.max.columns', 50)

api_key = '6ba1ec27f6281e7470d20ddef6b0000ddb901e31'
dart = OpenDartReader(api_key)

krx_df = fdr.StockListing('KRX')
symbols = krx_df['Symbol']

date= datetime.date(2019, 4, 1)
today = date.today()

In [5]:
info = pd.DataFrame(columns = ['Corporation', 'PER', 'PBR', 'ROE', 'Debt_R', 'Operating_R'])

for symbol in symbols[:300]:
    """모든 상장 기업들의 코드를 하나씩 돌며 info 데이터프레임에 저장"""
    
    true_year = today.year   # 사업보고서를 추출한 년도
    try:
        """금년->작년->재작년 사업보고서 순으로 탐색"""
        chart = dart.finstate(symbol, true_year, reprt_code='11011')
        if chart is None:
            chart = dart.finstate(symbol, true_year-1, reprt_code='11011')
            true_year -= 1
        if chart is None:
            chart = dart.finstate(symbol, true_year-1, reprt_code='11011')
            true_year -= 1
        if chart is None:
            continue
        
    except:
        """기업코드 데이터가 없음"""
        continue
    prev_chart = dart.finstate(symbol, true_year-1, reprt_code='11011')
    
    corp = dart.company(symbol)['stock_name']   # 주식 이름
    mc = today_mc(symbol, today.year)   # 시가총액
    if mc is None or prev_chart is None:
        continue
    pure = to_float(chart.loc[12, 'thstrm_amount'])   # 당기순이익
    asset =  to_float(chart.loc[2, 'thstrm_amount'])   # 총 자산
    equity = to_float(chart.loc[8, 'thstrm_amount'])   # 총 자본   
    debt = to_float(chart.loc[5, 'thstrm_amount'])   # 총 부채
    curr_operating =  to_float(chart.loc[10, 'thstrm_amount'])  # 당기 영업이익
    prev_operating = to_float(prev_chart.loc[10, 'thstrm_amount'])   # 전기 영업이익
    
    per = round(mc/pure, 2)
    pbr = round(mc/(asset-debt), 2)
    roe = round(100*pure/equity, 2)
    debt_r = round(100*debt/equity, 2)
    operating_r = round(100*(curr_operating-prev_operating)/prev_operating, 2)
    
    temp_list = [(corp, per, pbr, roe, debt_r, operating_r)]
    temp = pd.DataFrame(temp_list, columns = ['Corporation', 'PER', 'PBR', 'ROE', 'Debt_R', 'Operating_R'])
    info = info.append(temp, ignore_index=True)
    
info

,Corporation,PER,PBR,ROE,Debt_R,Operating_R
0,3S,-99.32,3.22,-3.24,48.83,-71.89
1,AJ네트웍스,4.41,0.54,12.13,419.16,-183.85
2,AK홀딩스,6.61,0.27,4.01,202.67,-51.31
3,APS홀딩스,-33.51,0.72,-2.14,38.61,192.70
4,AP시스템,43.59,3.61,8.28,268.87,-37.87
5,AP위성,366.06,3.18,0.87,24.78,708.53
6,BGF,32.84,0.38,1.15,8.45,-0.30
7,BGF리테일,19.75,4.80,24.31,248.93,3.75
8,BYC,8.08,0.52,6.40,59.76,9.59
9,CJ,8.39,0.19,2.22,176.30,13.26


In [6]:
"""필터링"""

condition = (info['PER'] < 5) & (info['PBR'] < 1) & (info['ROE'] > 15) & (info['Debt_R'] < 100) & (info['Operating_R'] > 20)
info[condition]

,Corporation,PER,PBR,ROE,Debt_R,Operating_R
47,HDC현대산업개발,4.29,0.8,18.54,97.58,73.46
